In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [2]:
### load the trained model, scaler pickle and onehot pickle
model = load_model('model.h5')

## Load the encoder and scaler
with open('onehot_encoder_geo.pkl','rb') as file:
  onehot_encoder_geo=pickle.load(file)

with open('label_encoder_gender.pkl','rb') as file:
  label_encoder_gender = pickle.load(file)

with open('scaler.pkl','rb') as file:
  scaler = pickle.load(file)

In [3]:
## Example input data
input_data = {
      'CreditScore':600,
      'Geography':'France',
      'Gender':'Male',
      'Age':40,
      'Tenure':3,
      'Balance':60000,
      'NumOfProducts':2,
      'HasCrCard':1,
      'IsActiveMember':1,
      'EstimatedSalary':50000

}

In [11]:
# one-hot encode 'Geography'
geo_encoded = onehot_encoder_geo.transform([[input_data['Geography']]])
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [13]:
# convert the input data into a dataframe
input_df= pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [14]:
## Encode categorical variables
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [15]:
# concatination with  one-hot encoded columns with input df
input_df = pd.concat([input_df.drop('Geography', axis=1),geo_encoded_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [16]:
# Scaling the input data
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [17]:
## Predict churn
prediction = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step


array([[0.03010364]], dtype=float32)

In [20]:
prediction_proba=prediction[0][0]
prediction_proba

0.030103635

In [22]:
if prediction_proba > 0.5:
  print("The customer is likely to churn.")
else:
  print('The customer is not likely to churn')

The customer is not likely to churn
